In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from scipy.spatial.distance import jaccard
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.preprocessing import StandardScaler


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/pancancer-8-dataset/8_cancers_merged.csv')


In [ ]:
df.columns = df.iloc[0]  # Set first row as column names
df = df[1:].reset_index(drop=True)  # Remove first row and reset index
df = df.rename(columns={'BRCA': 'cancer'})
df

# **Replace Ensemble IDs with actual Gene Names**

In [ ]:
genes = pd.read_csv('/kaggle/input/pancancer-8-dataset/genes.csv')
gene1 = 'DDX11L1'
genes = list(genes['DDX11L1']) 
genes.insert(0,'DDX11L1')
genes.append('cancer')
len(genes)

In [ ]:
df.columns = genes
df

# **Remove genes with 0 counts**

In [ ]:
# Remove columns where the sum is zero
temp = df.iloc[:,:-1]

temp = temp.apply(pd.to_numeric, errors='coerce')
df_filtered = temp.loc[:, temp.sum(axis=0) != 0]

# Optionally, reset the index if needed
df_filtered = df_filtered.reset_index(drop=True)

df_filtered


# **Filtering lowly expressed genes. In this case we have removed genes which are not expressed at least 50% of the samples.**# 

In [ ]:
# Step: Filtering Lowly Expressed Genes
# Keep genes that have log2(TPM) > 0 in at least 50% of the samples
df_filtered = df_filtered.loc[:, (df_filtered > 0).sum(axis=0) > 0.5 * df_filtered.shape[0]]  # Keep genes expressed in at least 50% of samples


In [ ]:
df_filtered

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Apply VarianceThreshold to the dataframe
selector = VarianceThreshold(threshold=0.01)  # Set an appropriate threshold

# Fit the selector and apply it to filter columns (features)
temp = df_filtered.loc[:, selector.fit(df_filtered).get_support()]

# Check the shape after variance thresholding
print(f"Shape after variance thresholding: {temp.shape}")

temp

In [ ]:
df_filtered = temp

# Z-score normalization

In [ ]:
# Assuming df_filtered is your DataFrame without the cancer label
# If your data contains NaN values, you may want to fill or drop them first
# df_filtered = df_filtered.fillna(df_filtered.mean())  # Optional step

# Z-score normalization (standardization)
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_filtered), columns=df_filtered.columns)



In [ ]:
df_normalized

In [ ]:
df_normalized['cancer'] = df['cancer']
df_normalized

In [ ]:
df_normalized.to_csv('Preprocessed_8_cancer_genes.csv')